In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/soccer/database.sqlite


In [2]:
## Read the liberaris
import sqlite3

In [3]:
## Map the dataset to a DB path and connect to sqlite base

path = "../input/soccer/"  
db = path + 'database.sqlite'
conn = sqlite3.connect(db)

In [6]:
#Query 1:- Types of leagues

sql = pd.read_sql(""" 
SELECT 
    id,
    name 
FROM league
GROUP BY name
""", conn)
sql

,id,name
0,1,Belgium Jupiler League
1,1729,England Premier League
2,4769,France Ligue 1
3,7809,Germany 1. Bundesliga
4,10257,Italy Serie A
5,13274,Netherlands Eredivisie
6,15722,Poland Ekstraklasa
7,17642,Portugal Liga ZON Sagres
8,19694,Scotland Premier League
9,21518,Spain LIGA BBVA


In [7]:
##Query 2:- League vs Country Name

sql = pd.read_sql(""" 
SELECT
    c.id,
    l.name as leagueName,
    c.name as countryName
FROM league l
JOIN country c 
ON c.id=l.id
GROUP BY c.name,l.name
""", conn)
sql

,id,leagueName,countryName
0,1,Belgium Jupiler League,Belgium
1,1729,England Premier League,England
2,4769,France Ligue 1,France
3,7809,Germany 1. Bundesliga,Germany
4,10257,Italy Serie A,Italy
5,13274,Netherlands Eredivisie,Netherlands
6,15722,Poland Ekstraklasa,Poland
7,17642,Portugal Liga ZON Sagres,Portugal
8,19694,Scotland Premier League,Scotland
9,21518,Spain LIGA BBVA,Spain


In [10]:
##Query 3:- Teams associated with the leagues (Country vs League vs Teams)

sql = pd.read_sql(""" 

SELECT 
    country_league_table.countryname,
    league_teams_table.LeagueName,
    league_teams_table.Teamintheleague 
FROM 
(SELECT
    l.name as LeagueName,
    t.team_long_name as TeamInTheLeague
FROM league l 
JOIN match m 
ON l.country_id = m.country_id
JOIN team t 
ON m.home_team_api_id = t.team_api_id
GROUP BY l.name,t.team_long_name) league_teams_table 
JOIN 
(SELECT 
    c.id,
    l.name as leagueName,
    c.name as countryName
FROM league l
JOIN country c 
ON c.id=l.id
GROUP BY c.name,l.name) country_league_table 
ON league_teams_table.leaguename=country_league_table.leaguename

""", conn)
sql


,countryName,LeagueName,TeamInTheLeague
0,Belgium,Belgium Jupiler League,Beerschot AC
1,Belgium,Belgium Jupiler League,Club Brugge KV
2,Belgium,Belgium Jupiler League,FCV Dender EH
3,Belgium,Belgium Jupiler League,KAA Gent
4,Belgium,Belgium Jupiler League,KAS Eupen
...,...,...,...
291,Switzerland,Switzerland Super League,FC Zürich
292,Switzerland,Switzerland Super League,Grasshopper Club Zürich
293,Switzerland,Switzerland Super League,Lugano
294,Switzerland,Switzerland Super League,Neuchâtel Xamax


In [12]:
 ##Query 4:-  Player Info
sql = pd.read_sql(""" 
SELECT  
    pa.player_api_id as Player_api_id,
    p.player_name as Player_Name, 
    strftime('%d-%m-%Y',p.birthday) as DOB,
    max(p.height) as Height,
    max(p.weight) as Weight,
    max(pa.overall_rating) as Rating,
    max(pa.potential) as Potenital,
    pa.preferred_foot as Preferred_Foot,
    pa.attacking_work_rate as Attacking_Work_Rate,
    pa.defensive_work_rate as Defensive_Work_Rate
FROM player p 
JOIN player_attributes pa 
ON p.player_api_id = pa.player_api_id 
AND p.player_fifa_api_id = pa.player_fifa_api_id
GROUP BY p.player_name
""", conn)
sql

,Player_api_id,Player_Name,DOB,Height,Weight,Rating,Potenital,Preferred_Foot,Attacking_Work_Rate,Defensive_Work_Rate
0,505942,Aaron Appindangoye,29-02-1992,182.88,187,67,71,right,medium,medium
1,155782,Aaron Cresswell,15-12-1989,170.18,146,74,80,left,medium,medium
2,162549,Aaron Doran,13-05-1991,170.18,163,71,78,right,medium,medium
3,30572,Aaron Galindo,08-05-1982,182.88,198,75,82,right,medium,high
4,23780,Aaron Hughes,08-11-1979,182.88,154,78,81,right,medium,medium
...,...,...,...,...,...,...,...,...,...,...
10843,36491,Zsolt Low,29-04-1979,180.34,154,68,74,left,None,_0
10844,35506,Zurab Khizanishvili,06-10-1981,185.42,172,78,79,right,None,_0
10845,39902,Zvjezdan Misimovic,05-06-1982,180.34,176,83,85,right,medium,low
10846,11833,de Oliveira Cleber Monteiro,23-05-1980,182.88,170,69,72,right,None,2


In [13]:
##Query 5:-  Away Team Info
sql = pd.read_sql(""" 
SELECT  
    match_api_id,
    away_team_api_id,
    team_long_name,
    team_short_name
FROM match m
JOIN team t 
ON t.team_api_id = m.away_team_api_id
GROUP BY match_api_id
""", conn)
sql

,match_api_id,away_team_api_id,team_long_name,team_short_name
0,483129,9830,FC Nantes,NAN
1,483130,7819,SM Caen,CAE
2,483131,9831,OGC Nice,NIC
3,483132,8689,FC Lorient,LOR
4,483133,9941,Toulouse FC,TOU
...,...,...,...,...
25974,2216668,9938,Dundee United,DUU
25975,2216669,8429,Hamilton Academical FC,HAM
25976,2216670,8649,Ross County FC,ROS
25977,2216671,9927,Motherwell,MOT


In [15]:
##Query 6:-  Home Team Info
sql = pd.read_sql(""" 
SELECT  
    match_api_id,
    home_team_api_id,
    team_long_name,
    team_short_name
FROM match m
JOIN team t 
ON t.team_api_id = m.home_team_api_id
GROUP BY match_api_id
""", conn)
sql

,match_api_id,home_team_api_id,team_long_name,team_short_name
0,483129,8583,AJ Auxerre,AUX
1,483130,9827,Girondins de Bordeaux,BOR
2,483131,9746,Le Havre AC,LEH
3,483132,8682,Le Mans FC,LEM
4,483133,9748,Olympique Lyonnais,LYO
...,...,...,...,...
25974,2216668,8597,Kilmarnock,KIL
25975,2216669,8426,Partick Thistle F.C.,PAR
25976,2216670,8485,Aberdeen,ABE
25977,2216671,9925,Celtic,CEL


In [24]:
##Query 7:-  Match By Match RAW
sql = pd.read_sql(""" 
WITH country_league_table AS
(
SELECT 
    c.id,
    l.name as leagueName,
    c.name as countryName
FROM league l
JOIN country c on c.id=l.id
GROUP BY c.name,l.name  
),

league_teams_table AS
(
SELECT
    l.name as LeagueName,
    t.team_long_name as TeamInTheLeague
FROM league l 
JOIN match m on l.country_id=m.country_id
JOIN team t on  m.home_team_api_id=t.team_api_id
GROUP BY l.name,t.team_long_name
),

country_team_league_table AS 
(
SELECT 
    country_league_table.countryname,
    league_teams_table.LeagueName,
    league_teams_table.Teamintheleague 
FROM league_teams_table 
JOIN country_league_table 
ON league_teams_table.leaguename = country_league_table.leaguename
),

stats AS 
(
SELECT 
    match_api_id,
    country_team_league_table.countryname,
    country_team_league_table.leaguename,
    home_team_goal,
    away_team_goal,
    season,
    date,
    stage
FROM match m 
JOIN team t 
ON t.team_api_id=m.home_team_api_id OR t.team_api_id=m.away_team_api_id
JOIN country_team_league_table 
ON t.team_long_name = country_team_league_table.teamintheleague
GROUP BY match_api_id
),

home_team AS 
(
SELECT  
    match_api_id,
    home_team_api_id,
    team_long_name,
    team_short_name
FROM match m
JOIN team t on t.team_api_id=m.home_team_api_id
GROUP BY match_api_id
),

away_team AS 
(
SELECT  
    match_api_id,
    away_team_api_id,
    team_long_name,
    team_short_name
FROM match m
JOIN team t on t.team_api_id=m.away_team_api_id
GROUP BY match_api_id
)

SELECT
    s.match_api_id,
    s.countryname,
    s.leaguename,
    ht.team_long_name as Home_Team_Long_Name,
    ht.team_short_name as Home_Team_Short_Name,
    at.team_long_name as Away_Team_Long_Name,
    at.team_short_name as Away_Team_Short_Name,
    s.season,
    strftime('%d-%m-%Y',s.date) as Date,
    s.stage,
    s.home_team_goal,
    s.away_team_goal,
    CASE 
        WHEN s.home_team_goal - s.away_team_goal > 0 then ht.team_short_name
        WHEN s.home_team_goal - s.away_team_goal < 0 then at.team_short_name ELSE 'Tie' END as Match_Winner
FROM stats s
JOIN home_team ht
ON s.match_api_id = ht.match_api_id 
JOIN away_team at 
ON s.match_api_id = at.match_api_id
""", conn)
sql.head()

,match_api_id,countryname,leaguename,Home_Team_Long_Name,Home_Team_Short_Name,Away_Team_Long_Name,Away_Team_Short_Name,season,Date,stage,home_team_goal,away_team_goal,Match_Winner
0,483129,France,France Ligue 1,AJ Auxerre,AUX,FC Nantes,NAN,2008/2009,09-08-2008,1,2,1,AUX
1,483130,France,France Ligue 1,Girondins de Bordeaux,BOR,SM Caen,CAE,2008/2009,09-08-2008,1,2,1,BOR
2,483131,France,France Ligue 1,Le Havre AC,LEH,OGC Nice,NIC,2008/2009,09-08-2008,1,1,0,LEH
3,483132,France,France Ligue 1,Le Mans FC,LEM,FC Lorient,LOR,2008/2009,09-08-2008,1,0,1,LOR
4,483133,France,France Ligue 1,Olympique Lyonnais,LYO,Toulouse FC,TOU,2008/2009,10-08-2008,1,3,0,LYO


In [20]:
##Query 8:Goals scored by home team vs away team Season on Season

sql = pd.read_sql(""" 
WITH league_teams_table AS
(
SELECT
    l.name as LeagueName,
    t.team_long_name as TeamInTheLeague
FROM league l 
JOIN match m on l.country_id=m.country_id
JOIN team t on  m.home_team_api_id=t.team_api_id
GROUP BY l.name,t.team_long_name
), 

country_league_table  AS
(
SELECT 
    c.id,
    l.name as leagueName,
    c.name as countryName
FROM league l
JOIN country c on c.id=l.id
GROUP BY c.name,l.name
),

country_team_league_table AS
(
SELECT 
    country_league_table.countryname,
    league_teams_table.LeagueName,
    league_teams_table.Teamintheleague 
FROM league_teams_table 
JOIN  country_league_table 
ON league_teams_table.leaguename = country_league_table.leaguename
), 

home_team AS 
(
SELECT  
    match_api_id,
    home_team_api_id,
    team_long_name,
    team_short_name
FROM match m
JOIN team t on t.team_api_id=m.home_team_api_id
GROUP BY match_api_id
), 

away_team AS
(
SELECT  
    match_api_id,
    away_team_api_id,
    team_long_name,
    team_short_name
FROM match m
JOIN team t on t.team_api_id=m.away_team_api_id
GROUP BY match_api_id
),  

league_stats AS
(
SELECT 
    match_api_id,
    country_team_league_table.countryname,
    country_team_league_table.leaguename,
    home_team_goal,
    away_team_goal,
    season,
    date,
    stage
FROM match m 
JOIN team t on t.team_api_id=m.home_team_api_id or t.team_api_id=m.away_team_api_id
JOIN country_team_league_table 
ON t.team_long_name = country_team_league_table.teamintheleague
GROUP BY match_api_id 
),

stats AS
(
SELECT
    a.match_api_id,
    a.countryname,
    a.leaguename,
    ht.team_long_name as Home_Team_Long_Name,
    ht.team_short_name as Home_Team_Short_Name,
    at.team_long_name as Away_Team_Long_Name,
    at.team_short_name as Away_Team_Short_Name,
    a.season,
    strftime('%d-%m-%Y',a.date) as Date,
    a.stage,
    a.home_team_goal,
    a.away_team_goal,
    CASE 
        WHEN a.home_team_goal - a.away_team_goal > 0 then ht.team_short_name
        WHEN a.home_team_goal - a.away_team_goal < 0 then at.team_short_name ELSE 'Tie' END as Match_Winner
FROM league_stats a
JOIN home_team  ht
ON a.match_api_id=ht.match_api_id 
JOIN away_team at
ON a.match_api_id=at.match_api_id
)

SELECT
    at_table.season,
    at_table.leaguename,
    ht_table.HT_goals as Home_Team_Goals,
    at_table.AT_goals as Away_Team_Goals
FROM 
(SELECT
    b.season,
    b.leaguename,
    SUM(home_team_goal) as HT_goals
FROM stats b
GROUP BY b.season,b.leaguename) ht_table 
JOIN
(SELECT
    b.season,
    b.leaguename,
    SUM(away_team_goal) as AT_goals
FROM stats b
GROUP BY b.season,b.leaguename) at_Table 
ON ht_table.leaguename = at_table.leaguename and ht_table.season = at_table.season


""", conn)
sql

,season,leaguename,Home_Team_Goals,Away_Team_Goals
0,2008/2009,Belgium Jupiler League,499,356
1,2008/2009,England Premier League,532,410
2,2008/2009,France Ligue 1,489,369
3,2008/2009,Germany 1. Bundesliga,520,374
4,2008/2009,Italy Serie A,578,410
...,...,...,...,...
83,2015/2016,Poland Ekstraklasa,341,294
84,2015/2016,Portugal Liga ZON Sagres,463,368
85,2015/2016,Scotland Premier League,343,307
86,2015/2016,Spain LIGA BBVA,615,428
